<a href="https://colab.research.google.com/github/hannape/IBAC-Biodiv/blob/master/04_Draw_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Wyrysowanie kilkukrotnego fitowania modelu na jednym wykresie 

Kod z I: Fig1_IBAC

In [0]:
from google.colab import drive
drive.mount('/content/drive')
from __future__ import print_function


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
import os
import matplotlib
from scipy import interp

In [0]:
# kolorki z tableau-20

tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),    
             (44, 160, 44), (152, 223, 138)]

for i in range(len(tableau20)):    
    r, g, b = tableau20[i]    
    tableau20[i] = (r / 255., g / 255., b / 255.)   

In [0]:
model_out = 'drive/My Drive/models/'                            # główny folder

rep_n = ["rep3V3/50epoch/","rep1V2/","rep5b/100epoch/"]         # duże reprezentacje
title_rys = "ROC for 3 representations (~60x148)"
rep_n = ["rep3b/","rep1b/50epoch/","rep5V2/nowa676/100epoch/"]  # małe reprezentacje
title_rys = "ROC for 3 representations (~60x64)"


font = {'family' : 'normal',
        'size'   : 16}

matplotlib.rc('font', **font)

base_fpr = np.linspace(0, 1, 101)

mean_auc=[]

plt.figure(figsize=(7,7))

for j in (0,np.shape(rep_n)[0]):
  fpr = []
  tpr = []
  tprs = []
  runs = []
  auc_sum = 0
  auc_nr = 0
  
  runs += [each for each in os.listdir (model_out+rep_n[j]) if each.endswith('.npz')]
  print(runs)

  for i in runs:
      a = np.load(model_out + rep_n[j] + str(i))
      tpr = a["tpr"]
      fpr = a["fpr"]
      roc_auc = metrics.auc(a["fpr"], a["tpr"])
      print(roc_auc)
      if roc_auc>0.51:
        #plt.plot(fpr, tpr, 'b', alpha=0.15)
        tpr = interp(base_fpr, fpr, tpr)
        tpr[0] = 0.0
        tprs.append(tpr)
        auc_sum += roc_auc
        auc_nr += 1
  print(auc_sum)
  print(auc_nr)
  mean_auc.append(auc_sum/auc_nr)
  print(mean_auc)
  tprs = np.array(tprs)
  mean_tprs = tprs.mean(axis=0)
  std = tprs.std(axis=0)

  tprs_upper = np.minimum(mean_tprs + std, 1)
  tprs_lower = mean_tprs - std

  plt.plot(base_fpr, mean_tprs, color = tableau20[j*2])
  plt.fill_between(base_fpr, tprs_lower, tprs_upper, color=tableau20[2*j+1], alpha=0.25)
  
print(mean_auc)
#plt.plot([0, 1], [0, 1],'r--')
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.title(title_rys)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.axes().set_aspect('equal', 'datalim')
plt.legend(["mel-spectrogram, mean AUC: %.3f, epochs: %.0f" % (mean_auc[0], 50),
            "spectrogram, mean AUC: %.3f, epochs: %.0f " % (mean_auc[1], 30),
            "multitaper, mean AUC: %.3f, epochs: %.0f" % (mean_auc[2], 100)])
plt.show()